In [142]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# ___library_import_statements___
import pandas as pd
# make pandas to print dataframes nicely
pd.set_option('expand_frame_repr', False)  
#import pandas_datareader.data as web
import matplotlib.pyplot as plt
#import datetime


In [143]:
#newest yahoo API 
!pip install yfinance
import yfinance as yf
%matplotlib inline

In [156]:
# ___variables___
ticker = '0002.HK'                                               

In [157]:
# yahoo gives only daily historical data
def get_data(ticker):
    connected = False
    while not connected:
        try:
            df = yf.download(ticker, period="5y", interval="1d", group_by = 'ticker', auto_adjust = False)
            connected = True
            print('connected to yahoo')
        except Exception as e:
            print("type error: " + str(e))
            time.sleep( 5 )
            pass   
    return df    

# run the function
df = get_data(ticker)   

In [158]:
# optional printing
print(df.head(7))
print(df.tail(7))

In [159]:
# resample with dictionary
# each column has to be handled differently
agg_dict = {'Open': 'first',
          'High': 'max',
          'Low': 'min',
          'Close': 'last',
          'Adj Close': 'last',
          'Volume': 'mean'}

r_df = df.resample('W').agg(agg_dict)

In [160]:
# optional printing
print(df.head(15))
print('---------')
print(r_df.tail(7))

In [161]:
def plot_OHLC(data, ticker, window):

    #to avoid matplotlib future warning
    from pandas.plotting import register_matplotlib_converters
    register_matplotlib_converters()    
    
    plt.figure(figsize=(15,5))
    plt.title('{} price data to {} timeframe'.format(ticker, window))
    plt.plot(data['Open'])
    plt.plot(data['High'])
    plt.plot(data['Low'])
    plt.plot(data['Close'])
    plt.plot(data['Adj Close'])
    plt.legend()
    plt.show()

In [162]:
#plot_OHLC(df, ticker, 'daily')
plot_OHLC(r_df, ticker, 'weekly')

In [163]:
def computeRSI(data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

In [164]:
df['RSI'] = computeRSI(df['Close'], 14)
r_df['RSI'] = computeRSI(r_df['Close'], 14)

In [165]:
# optional printing
print(r_df.head())
print(r_df.tail())

In [166]:
def plot_price_and_RSI(dataframe, ticker, window):

    # plot price
    plt.figure(figsize=(15,5))
    plt.plot(dataframe['Adj Close'])
    plt.title('{} price data (Adj Close) to {} timeframe'.format(ticker, window))
    plt.show()

    # plot corresponding RSI values and significant levels
    plt.figure(figsize=(15,5))
    plt.title('RSI chart')
    plt.title('{} RSI chart to {} timeframe'.format(ticker, window))
    plt.plot(dataframe['RSI'])

    plt.axhline(0, linestyle='--', alpha=0.1)
    plt.axhline(20, linestyle='--', alpha=0.5)
    plt.axhline(30, linestyle='--')

    plt.axhline(70, linestyle='--')
    plt.axhline(80, linestyle='--', alpha=0.5)
    plt.axhline(100, linestyle='--', alpha=0.1)
    plt.show()

In [167]:
plot_price_and_RSI(r_df, ticker, 'weekly')